# Геолокация сообщений

#### Импорт библиотек

In [ ]:
import warnings

warnings.simplefilter("ignore")

import os
import sys

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

import sloyka
from sloyka import Geocoder

#### Загрузка данных

In [ ]:
import pandas as pd

df = pd.read_csv('/home/gk/vscode/SOIKA_2/sloyka/sloyka/sample_data/sample_data.csv', index_col=0)

In [ ]:
df.head()

#### Запуск геолокатора со встроенным геокодером

In [ ]:
result = Geocoder().run(df[:20], text_column='Текст комментария')

In [ ]:
result[1]

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.reset_option('display.width', None)
pd.set_option('display.width', 2000)


In [ ]:
mask = (result.Score.isna())&(~result.Street.isna())
result[mask]
# result.to_csv('C:\Users\USER\SOIKA\сохры.csv', index=False)

In [ ]:
from shapely.geometry import Point

result['geometry'] = result.Location.progress_apply(lambda x: Point(x.longitude, x.latitude))

#### Визаулизация полученных сообщений

In [ ]:
result.head(3)

In [ ]:
# result.explore()